Ripped from https://github.com/mees/calvin/blob/main/RL_with_CALVIN.ipynb

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('/home/grail/willaria_research/hobbes/calvin_env'))
if module_path not in sys.path:
    sys.path.append(module_path)
print(sys.path)

['/home/grail/willaria_research/hobbes', '', '/home/grail/interbotix_ws/devel/lib/python3/dist-packages', '/opt/ros/noetic/lib/python3/dist-packages', '/home/grail/miniconda3/envs/calvin_conda_env/lib/python38.zip', '/home/grail/miniconda3/envs/calvin_conda_env/lib/python3.8', '/home/grail/miniconda3/envs/calvin_conda_env/lib/python3.8/lib-dynload', '/home/grail/.local/lib/python3.8/site-packages', '/home/grail/miniconda3/envs/calvin_conda_env/lib/python3.8/site-packages', '/home/grail/willaria_research/hobbes/calvin_models', '/home/grail/willaria_research/hobbes/calvin_env/tacto', '/home/grail/willaria_research/hobbes/calvin_env']


In [2]:
import time
import hydra
import numpy as np
import cv2
from hydra import initialize, compose
from calvin_env.envs.play_table_env import PlayTableSimEnv

pybullet build time: May 20 2022 19:44:17


In [3]:

with initialize(config_path="./calvin_env/conf/"):
  cfg = compose(config_name="config_data_collection.yaml", overrides=["cameras=static_and_gripper"])
  cfg.env["use_egl"] = False
  cfg.env["show_gui"] = False
  cfg.env["use_vr"] = False
  cfg.env["use_scene_info"] = True
  print(cfg.env)

{'_target_': 'calvin_env.envs.play_table_env.PlayTableSimEnv', '_recursive_': False, 'cameras': '${cameras}', 'seed': 0, 'bullet_time_step': 240.0, 'use_vr': False, 'show_gui': False, 'robot_cfg': '${robot}', 'scene_cfg': '${scene}', 'use_scene_info': True, 'use_egl': False, 'control_freq': 30}


/home/grail/miniconda3/envs/calvin_conda_env/lib/python3.8/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config_data_collection.yaml': Defaults list is missing `_self_`. See https://hydra.cc/docs/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)
/home/grail/miniconda3/envs/calvin_conda_env/lib/python3.8/site-packages/hydra/core/default_element.py:122: UserWarning: In 'vr_input/vr_controller/vive': Usage of deprecated keyword in package header '# @package _group_'.
See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/changes_to_package_header for more information
  deprecation_warning(


This cell below runs code to show images from some static viewpoint over a few random steps. If it says "Environment View is ready", just click on that.

In [4]:
env = hydra.utils.instantiate(cfg.env)
observation = env.reset()
#The observation is given as a dictionary with different values
print(observation.keys())
for i in range(5):
  # The action consists in a pose displacement (position and orientation)
  action_displacement = np.random.uniform(low=-10, high=10, size=6)
  # And a binary gripper action, -1 for closing and 1 for oppening
  action_gripper = np.random.choice([-1, 1], size=1)
  action = np.concatenate((action_displacement, action_gripper), axis=-1)
  print(action)
  observation, reward, done, info = env.step(action)
  rgb = env.render(mode="rgb_array")[:,:,::-1]
  cv2.imshow("Environment View", rgb)
  cv2.waitKey(0)
cv2.destroyAllWindows()

WARNING uncommitted modified files: calvin_env/envs/play_table_env.py
dict_keys(['rgb_obs', 'depth_obs', 'robot_obs', 'scene_obs'])
[ 7.08682948  5.99171432 -7.03224702 -5.49318887 -1.70009013 -7.65507016
  1.        ]


QObject::moveToThread: Current thread (0x50c5650) is not the object's thread (0x89bf6f0).
Cannot move to target thread (0x50c5650)

QObject::moveToThread: Current thread (0x50c5650) is not the object's thread (0x89bf6f0).
Cannot move to target thread (0x50c5650)

QObject::moveToThread: Current thread (0x50c5650) is not the object's thread (0x89bf6f0).
Cannot move to target thread (0x50c5650)

QObject::moveToThread: Current thread (0x50c5650) is not the object's thread (0x89bf6f0).
Cannot move to target thread (0x50c5650)

QObject::moveToThread: Current thread (0x50c5650) is not the object's thread (0x89bf6f0).
Cannot move to target thread (0x50c5650)

QObject::moveToThread: Current thread (0x50c5650) is not the object's thread (0x89bf6f0).
Cannot move to target thread (0x50c5650)

QObject::moveToThread: Current thread (0x50c5650) is not the object's thread (0x89bf6f0).
Cannot move to target thread (0x50c5650)

QObject::moveToThread: Current thread (0x50c5650) is not the object's thread

[-9.8862837  -0.61783161 -3.25827649  0.97254457  6.86163348 -5.2310717
 -1.        ]
[-0.73251364 -3.01883638  6.2745267   4.060537    6.28361819 -3.04713221
 -1.        ]
[-2.31676275  7.06237823  2.21462613  7.81254149 -8.9763565  -2.65253882
 -1.        ]
[-9.5982261   1.53404545  8.97277979  5.87563272  9.09013324  9.79839947
 -1.        ]


In [5]:
%cd /home/grail/willaria_research/hobbes/dataset/calvin_debug_dataset/training

/home/grail/willaria_research/hobbes/dataset/calvin_debug_dataset/training


In [6]:
lang = np.load("lang_annotations/auto_lang_ann.npy", allow_pickle=True)
lang = dict(enumerate(lang.flatten(), 1))[1]
lang

{'language': {'ann': ['move the light switch to turn on the yellow light',
   'sweep the pink block to the right',
   'place the block in the sliding cabinet',
   'pick up the red block from the table',
   'in the slider grasp the blue block',
   'slide down the switch',
   'pick up the red block on the table',
   'place in slider',
   'turn off the light bulb'],
  'task': ['turn_on_lightbulb',
   'push_pink_block_right',
   'place_in_slider',
   'lift_red_block_table',
   'lift_blue_block_slider',
   'turn_off_lightbulb',
   'lift_red_block_table',
   'place_in_slider',
   'turn_off_lightbulb'],
  'emb': array([[[ 0.52925   ,  0.09874008,  0.15309368, ...,  0.6170549 ,
           -0.14766425,  0.21653481]],
  
         [[ 0.11232971, -0.14778604,  0.38895708, ...,  0.40465486,
            0.4906702 ,  0.2589114 ]],
  
         [[-0.04571903, -0.0061444 ,  0.27441305, ...,  0.07715372,
           -0.02765487,  0.1338684 ]],
  
         ...,
  
         [[-0.2222732 ,  0.02141502,  0.29

In [7]:
lang['language']['emb'].shape

(9, 1, 384)

In [8]:
len(lang['info']['indx'])

9

In [9]:
print(lang['language']['ann'][5])
print(lang['info']['indx'][5])

slide down the switch
(360571, 360635)


In [11]:
def display_frames(frames, title="", ms_per_frame=50):
    for i in range(len(frames)):
        if i == 1:
            _ = cv2.waitKey(0)
        cv2.imshow(title, frames[i])
        
        key = cv2.waitKey(50)#pauses for 3 seconds before fetching next image
        if key == 27:#if ESC is pressed, exit loop
            cv2.destroyAllWindows()
            break
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def collect_frames(start, stop):
    result = []
    for i in range(start, stop):
        
        result.append(np.load("episode_0" + str(i) + ".npz")["rgb_static"][:,:,::-1])
    return result


In [ ]:
task_idx = 0
frames = collect_frames(lang['info']['indx'][task_idx][0], lang['info']['indx'][task_idx][1])
display_frames(frames, "Static (200 x 200)")

In [16]:
lang['info']['indx'][task_idx][0]

360575

In [17]:
ep = np.load("episode_0360575.npz")

In [18]:
dict(ep)['scene_obs']

array([ 2.77312593e-01,  1.77498972e-01,  1.18748532e-19,  8.80000000e-02,
        1.00000000e+00,  1.00000000e+00,  6.21741856e-02, -1.05260515e-01,
        4.59990009e-01, -1.57079633e+00,  1.01304853e-09, -1.54320127e+00,
       -2.39174288e-01, -8.85086970e-02,  4.59965526e-01,  1.92448627e-05,
        3.39719465e-03, -1.62377881e+00,  1.62451444e-01, -1.15017202e-01,
        4.72010064e-01, -6.46464608e-02,  5.99302667e-02, -1.65834741e+00])

In [ ]:

[ 2.77312593e-01  1.77498972e-01  1.18748532e-19  8.80000000e-02
  1.00000000e+00  1.00000000e+00  6.21741856e-02 -1.05260515e-01
  4.59990009e-01 -1.57079633e+00  1.01304853e-09 -1.54320127e+00
 -2.39174288e-01 -8.85086970e-02  4.59965526e-01  1.92448627e-05
  3.39719465e-03 -1.62377881e+00  1.62451444e-01 -1.15017202e-01
  4.72010064e-01 -6.46464608e-02  5.99302667e-02 -1.65834741e+00]

In [20]:

with initialize(config_path="./calvin_env/conf/"):
  cfg = compose(config_name="config_data_collection.yaml", overrides=["cameras=static_and_gripper"])
  cfg.env["use_egl"] = False
  cfg.env["show_gui"] = False
  cfg.env["use_vr"] = False
  cfg.env["use_scene_info"] = True
  print(cfg.env)

{'_target_': 'calvin_env.envs.play_table_env.PlayTableSimEnv', '_recursive_': False, 'cameras': '${cameras}', 'seed': 0, 'bullet_time_step': 240.0, 'use_vr': False, 'show_gui': False, 'robot_cfg': '${robot}', 'scene_cfg': '${scene}', 'use_scene_info': True, 'use_egl': False, 'control_freq': 30}


/home/grail/miniconda3/envs/calvin_env/lib/python3.8/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config_data_collection.yaml': Defaults list is missing `_self_`. See https://hydra.cc/docs/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)
/home/grail/miniconda3/envs/calvin_env/lib/python3.8/site-packages/hydra/core/default_element.py:122: UserWarning: In 'vr_input/vr_controller/vive': Usage of deprecated keyword in package header '# @package _group_'.
See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/changes_to_package_header for more information
  deprecation_warning(


In [22]:
env = hydra.utils.instantiate(cfg.env)
observation = env.reset()
#The observation is given as a dictionary with different values
print(observation.keys())
for i in range(5):
  # The action consists in a pose displacement (position and orientation)
  action_displacement = np.random.uniform(low=-1, high=1, size=6)
  # And a binary gripper action, -1 for closing and 1 for oppening
  action_gripper = np.random.choice([-1, 1], size=1)
  action = np.concatenate((action_displacement, action_gripper), axis=-1)
  observation, reward, done, info = env.step(action)
  print(observation['scene_obs'])
  rgb = env.render(mode="rgb_array")[:,:,::-1]
  cv2_imshow(rgb)

disconnecting id 1 from server
dict_keys(['rgb_obs', 'depth_obs', 'robot_obs', 'scene_obs'])
[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.44560290e-19
  0.00000000e+00  0.00000000e+00  9.89883188e-03 -1.35075945e-01
  4.59971892e-01  5.11323035e-05 -8.29153911e-04 -7.30876980e-01
 -1.63031718e-01  9.80176982e-02  4.60492255e-01 -3.80765860e-03
 -8.24435498e-04 -2.29269375e+00  7.98186152e-02  8.67648292e-02
  4.60475298e-01 -4.54503815e-03  8.67976304e-05  2.72751209e+00]


NameError: name 'cv2_imshow' is not defined

# Exploration of task_D_D dataset

In [22]:
%cd /home/grail/willaria_research/hobbes/dataset/task_D_D/training

/home/grail/willaria_research/hobbes/dataset/task_D_D/training


In [46]:
lang = np.load("lang_annotations/auto_lang_ann.npy", allow_pickle=True)
lang = dict(enumerate(lang.flatten(), 1))[1]
lang

{'language': {'ann': ['move the door to the left side',
   'slide the door to the left side',
   'slide down the switch',
   'toggle the button to turn on the green light',
   'toggle the light switch to turn on the yellow light',
   'push the switch upwards',
   'push down the button to turn on the led',
   'open the cabinet drawer',
   'grasp the drawer handle and open it',
   'move up the switch',
   'pull the handle of the drawer',
   'move the sliding door to the left',
   'put the block in the drawer',
   'toggle the light switch to turn off the light bulb',
   'move the door to the right side',
   'turn on the yellow light',
   'grasp the blue block and lift it up',
   'toggle the button to turn off the led light',
   'sweep the block into the drawer',
   'push the sliding door to the right',
   'take the red block and rotate it left',
   'move the door to the left side',
   'close the cabinet drawer',
   'grasp the red block from the drawer',
   'remove the top block',
   'turn

In [42]:
# Count up number of each task
task_counts = {}
for task in lang['language']['task']:
    task_counts[task] = task_counts.get(task, 0) + 1
for task in task_counts:
    print(task_counts[task], '', task)

151  move_slider_left
152  turn_off_lightbulb
150  turn_on_led
150  turn_on_lightbulb
152  open_drawer
152  place_in_drawer
152  move_slider_right
152  lift_blue_block_table
150  turn_off_led
150  push_into_drawer
150  rotate_red_block_left
151  close_drawer
151  lift_red_block_drawer
151  unstack_block
151  lift_red_block_slider
150  lift_red_block_table
150  rotate_pink_block_right
151  rotate_pink_block_left
151  lift_pink_block_slider
151  lift_pink_block_table
151  place_in_slider
154  lift_blue_block_slider
150  lift_pink_block_drawer
150  rotate_red_block_right
150  lift_blue_block_drawer
151  stack_block
150  push_pink_block_left
150  rotate_blue_block_left
150  push_red_block_left
150  rotate_blue_block_right
150  push_blue_block_right
150  push_pink_block_right
150  push_red_block_right
150  push_blue_block_left


In [43]:
# Get all time sequences corresponding to a specific task
target_task = 'turn_on_led'
target_task_indxs = []
for (task, indx) in zip(lang['language']['task'], lang['info']['indx']):
    if task == target_task:
        target_task_indxs.append(indx)
print(target_task_indxs)

[(340730, 340794), (575627, 575691), (569905, 569942), (160666, 160699), (431711, 431757), (84090, 84154), (315208, 315272), (497325, 497389), (75570, 75616), (356918, 356982), (292313, 292359), (143853, 143917), (288614, 288678), (170856, 170920), (607015, 607079), (146222, 146286), (69959, 70023), (193395, 193459), (115014, 115078), (459816, 459854), (60792, 60831), (115002, 115066), (59895, 59959), (395936, 396000), (208796, 208860), (525529, 525593), (608215, 608279), (278781, 278828), (195013, 195077), (460821, 460865), (435204, 435247), (181562, 181606), (564883, 564947), (148845, 148909), (265150, 265214), (303287, 303328), (113655, 113719), (147024, 147088), (602184, 602248), (340733, 340797), (537578, 537620), (599736, 599800), (190245, 190309), (385094, 385132), (581332, 581396), (169579, 169643), (456312, 456352), (392674, 392721), (585658, 585722), (479151, 479215), (146217, 146281), (80479, 80543), (116915, 116963), (331471, 331535), (366452, 366516), (100240, 100304), (51

In [44]:
# break here
a a aa a =a =a =a=a== 

SyntaxError: invalid syntax (1958689592.py, line 2)

In [13]:
lang["language"]["ann"]

['move the light switch to turn on the yellow light',
 'sweep the pink block to the right',
 'place the block in the sliding cabinet',
 'pick up the red block from the table',
 'in the slider grasp the blue block',
 'slide down the switch',
 'pick up the red block on the table',
 'place in slider',
 'turn off the light bulb']

In [14]:
lang["language"]["task"]

['turn_on_lightbulb',
 'push_pink_block_right',
 'place_in_slider',
 'lift_red_block_table',
 'lift_blue_block_slider',
 'turn_off_lightbulb',
 'lift_red_block_table',
 'place_in_slider',
 'turn_off_lightbulb']

In [15]:
lang["language"]["emb"].shape # 9 examples (corresponding to ann?), each is a 384 dim vector

(9, 1, 384)

In [16]:
episode = np.load("episode_0358482.npz")

In [17]:
for key in episode.keys():
    print(key + " : ", episode[key].shape)

actions :  (7,)
rel_actions :  (7,)
robot_obs :  (15,)
scene_obs :  (24,)
rgb_static :  (200, 200, 3)
rgb_gripper :  (84, 84, 3)
rgb_tactile :  (160, 120, 6)
depth_static :  (200, 200)
depth_gripper :  (84, 84)
depth_tactile :  (160, 120, 2)


In [18]:
cv2.imshow("Static (200 x 200)", episode["rgb_static"])
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imshow("Gripper (84 x 84)", episode["rgb_gripper"]) #They have to have different names, or else override!
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imshow("Depth Static (200 x 200)", episode["depth_static"]) 
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imshow("Depth Gripper (84 x 84)", episode["depth_gripper"])
cv2.waitKey(0)
cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x7c00560) is not the object's thread (0x2fcb6b0).
Cannot move to target thread (0x7c00560)

QObject::moveToThread: Current thread (0x7c00560) is not the object's thread (0x2fcb6b0).
Cannot move to target thread (0x7c00560)

QObject::moveToThread: Current thread (0x7c00560) is not the object's thread (0x2fcb6b0).
Cannot move to target thread (0x7c00560)

QObject::moveToThread: Current thread (0x7c00560) is not the object's thread (0x2fcb6b0).
Cannot move to target thread (0x7c00560)

QObject::moveToThread: Current thread (0x7c00560) is not the object's thread (0x2fcb6b0).
Cannot move to target thread (0x7c00560)

QObject::moveToThread: Current thread (0x7c00560) is not the object's thread (0x2fcb6b0).
Cannot move to target thread (0x7c00560)

QObject::moveToThread: Current thread (0x7c00560) is not the object's thread (0x2fcb6b0).
Cannot move to target thread (0x7c00560)

QObject::moveToThread: Current thread (0x7c00560) is not the object's thread

In [15]:
episode = np.load("episode_0358482.npz")
cv2.imshow("Static (200 x 200)", episode["rgb_static"])
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imshow("Gripper (84 x 84)", episode["rgb_gripper"]) #They have to have different names, or else override!
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imshow("Depth Static (200 x 200)", episode["depth_static"]) 
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imshow("Depth Gripper (84 x 84)", episode["depth_gripper"])
cv2.waitKey(0)
cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x74083e0) is not the object's thread (0x2254870).
Cannot move to target thread (0x74083e0)

QObject::moveToThread: Current thread (0x74083e0) is not the object's thread (0x2254870).
Cannot move to target thread (0x74083e0)

QObject::moveToThread: Current thread (0x74083e0) is not the object's thread (0x2254870).
Cannot move to target thread (0x74083e0)

QObject::moveToThread: Current thread (0x74083e0) is not the object's thread (0x2254870).
Cannot move to target thread (0x74083e0)

QObject::moveToThread: Current thread (0x74083e0) is not the object's thread (0x2254870).
Cannot move to target thread (0x74083e0)

QObject::moveToThread: Current thread (0x74083e0) is not the object's thread (0x2254870).
Cannot move to target thread (0x74083e0)

QObject::moveToThread: Current thread (0x74083e0) is not the object's thread (0x2254870).
Cannot move to target thread (0x74083e0)

QObject::moveToThread: Current thread (0x74083e0) is not the object's thread

In [17]:
episode = np.load("episode_0358482.npz")
cv2.imshow("Static (200 x 200)", episode["rgb_static"])
cv2.waitKey(0)
cv2.destroyAllWindows()
episode = np.load("episode_0358483.npz")
cv2.imshow("Static (200 x 200)", episode["rgb_static"])
cv2.waitKey(0)
cv2.destroyAllWindows()
episode = np.load("episode_0358484.npz")
cv2.imshow("Static (200 x 200)", episode["rgb_static"])
cv2.waitKey(0)
cv2.destroyAllWindows()
episode = np.load("episode_0358485.npz")
cv2.imshow("Static (200 x 200)", episode["rgb_static"])
cv2.waitKey(0)
cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x74083e0) is not the object's thread (0x2254870).
Cannot move to target thread (0x74083e0)

QObject::moveToThread: Current thread (0x74083e0) is not the object's thread (0x2254870).
Cannot move to target thread (0x74083e0)

QObject::moveToThread: Current thread (0x74083e0) is not the object's thread (0x2254870).
Cannot move to target thread (0x74083e0)

QObject::moveToThread: Current thread (0x74083e0) is not the object's thread (0x2254870).
Cannot move to target thread (0x74083e0)

QObject::moveToThread: Current thread (0x74083e0) is not the object's thread (0x2254870).
Cannot move to target thread (0x74083e0)

QObject::moveToThread: Current thread (0x74083e0) is not the object's thread (0x2254870).
Cannot move to target thread (0x74083e0)

QObject::moveToThread: Current thread (0x74083e0) is not the object's thread (0x2254870).
Cannot move to target thread (0x74083e0)

QObject::moveToThread: Current thread (0x74083e0) is not the object's thread